## Import thư viện

In [1]:
import sys

# gensim modules
from gensim import utils
from gensim.models.doc2vec import TaggedDocument
from gensim.models import Doc2Vec

#glob
import glob

# numpy
import numpy as np

#pandas
import pandas as pd

# classifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

# random, itertools, matplotlib
import random

import multiprocessing

import timeit
import itertools

C:\Users\ADMIN\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


## Class đọc từng dòng 'content' trong file csv

In [2]:
class LabeledContent(object):
    def __init__(self, sources):
        self.sources = sources

        flipped = {}

        # make sure that keys are unique
        for key, value in sources.items():
            if value not in flipped:
                flipped[value] = [key]
            else:
                raise Exception('Non-unique prefix encountered')

    def to_array(self):
        self.sentences = []
        item_no = 0
        for source, prefix in self.sources.items():
            allFiles = glob.glob(source + "/*.csv")
            for file_ in allFiles:
                df = pd.read_csv(file_)
                content = df['content']
                title = df['title']
                print('number documents: ', len(content))
                for row in content:
                    self.sentences.append(TaggedDocument(utils.to_unicode(row).split(), [prefix + '_%s' % item_no]))
                    item_no = item_no + 1
        return self.sentences, title

    def sentences_perm(self):
        shuffled = list(self.sentences)
        random.shuffle(shuffled)
        return shuffled

## Xét đường dẫn tới dataset

In [3]:
path = 'C:\\Users\\ADMIN\\Desktop\\Python Tutorial 1\\data\\a4'
title_path = 'news_title_d2v.csv'
#path = 'C:\\Users\\ADMIN\\Desktop\\Python Tutorial 1\\Doc2vec\\data\\a1'
model_name = 'halleluja.d2v'
_epochs = '1'
cores = '7'

In [4]:
sources = {path: 'SAMPLE', }
print('Load data ...')
start = timeit.default_timer()
sentences = LabeledContent(sources)
stop = timeit.default_timer()
print('Done! time: ', stop - start, ' (s)')

Load data ...
Done! time:  4.938259899433325e-05  (s)


## Xét tham số cho model, build vocabulary

In [5]:
model = Doc2Vec(min_count=5, window=10, vector_size=400, sample=1e-4, negative=5, workers=int(cores), dm=0)

In [6]:
print('Tokenize ...')
start = timeit.default_timer()
token,title = sentences.to_array()
stop = timeit.default_timer()
print('Done! time: ', stop - start, ' (s)')

Tokenize ...
number documents:  9944
Done! time:  1.0070684276896538  (s)


In [7]:
print('Saving title ...')
start = timeit.default_timer()
title.to_csv(title_path)
stop = timeit.default_timer()
print('Done! time: ', stop - start, ' (s)')

Saving title ...
Done! time:  0.053092614891578904  (s)


In [8]:
print('Build vocabulary ...')
start = timeit.default_timer()
model.build_vocab(token)
stop = timeit.default_timer()
print('Done! time: ', stop - start, ' (s)')

Build vocabulary ...
Done! time:  1.38719513158685  (s)


## Train model

In [9]:
print('Train model ...')
start = timeit.default_timer()
model.train(sentences.sentences_perm(), total_examples=model.corpus_count, epochs=int(_epochs))
stop = timeit.default_timer()
print('Done! time: ', stop - start, ' (s)')

Train model ...
Done! time:  2.850964106356688  (s)


## Save model

In [10]:
print('Save model ...')
model.save(model_name)
print('Done!')

Save model ...
Done!
